In [35]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import shutil
import os

In [3]:
!wget google.com

--2025-02-15 19:12:07--  http://google.com/
Resolving google.com (google.com)... 216.239.38.120, 2001:4860:4802:32::78
Connecting to google.com (google.com)|216.239.38.120|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://www.google.com/ [following]
--2025-02-15 19:12:08--  http://www.google.com/
Resolving www.google.com (www.google.com)... 216.239.38.120, 2001:4860:4802:32::78
Reusing existing connection to google.com:80.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html’

index.html              [ <=>                ]  18.16K  --.-KB/s    in 0.03s   

2025-02-15 19:12:08 (533 KB/s) - ‘index.html’ saved [18592]



In [4]:
!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"

--2025-02-15 19:12:13--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 184.86.118.65, 2a02:26f0:8d00:d8d::317f, 2a02:26f0:8d00:d82::317f
Connecting to download.microsoft.com (download.microsoft.com)|184.86.118.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_5340.zip’

kagglecatsanddogs_5 100%[===================>] 786.67M  3.78MB/s    in 10m 23s 

2025-02-15 19:22:36 (1.26 MB/s) - ‘kagglecatsanddogs_5340.zip’ saved [824887076/824887076]



In [7]:
!unzip -xq './kagglecatsanddogs_5340.zip'

In [9]:
count=0
for path in glob('./PetImages/*/*'):
  if not os.path.getsize(path):
    count+=1
    os.remove(path)
count

2

In [12]:
folder='./dataset'

train_cats_dir=f'{folder}/training/cats'
train_dogs_dir=f'{folder}/training/dogs'


test_cats_dir=f'{folder}/testing/cats'
test_dogs_dir=f'{folder}/testing/dogs'

folders=[
    train_cats_dir,
    train_dogs_dir,
    test_cats_dir,
    test_dogs_dir,
]

for f in folders:
  os.makedirs(f)

In [13]:
all_cats=sorted(glob('./PetImages/Cat/*'))
all_dogs=sorted(glob('./PetImages/Dog/*'))
len(all_cats),len(all_dogs)

(12500, 12500)

In [14]:
dogs_train,dogs_test=train_test_split(all_dogs,test_size=0.1,shuffle=True,random_state=1403)
len(dogs_train),len(dogs_test)

(11250, 1250)

In [15]:
cats_train,cats_test=train_test_split(all_cats,test_size=0.1,shuffle=True,random_state=1403)
len(cats_train),len(cats_test)

(11250, 1250)

In [16]:
dogs_train[0]

'./PetImages/Dog/9388.jpg'

In [17]:
for name in dogs_train:
  shutil.copy(name,os.path.join(train_dogs_dir,name.split('/')[-1]))

In [18]:
len(os.listdir(train_dogs_dir))

11250

In [20]:
for name in dogs_test:
  shutil.copy(name,os.path.join(test_dogs_dir,name.split('/')[-1]))


for name in cats_train:
  shutil.copy(name,os.path.join(train_cats_dir,name.split('/')[-1]))

for name in cats_test:
  shutil.copy(name,os.path.join(test_cats_dir,name.split('/')[-1]))

In [21]:
len(os.listdir(test_dogs_dir)),len(os.listdir(train_cats_dir)),len(os.listdir(test_cats_dir))

(1250, 11250, 1250)

In [22]:
training_dir='./dataset/training'
train_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.2)
train_generator = train_datagen.flow_from_directory(training_dir,batch_size=64,class_mode='binary',target_size=(150,150))

Found 22498 images belonging to 2 classes.


In [23]:
testing_dir='./dataset/testing'
test_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.2)
test_generator = test_datagen.flow_from_directory(testing_dir,batch_size=64,class_mode='binary',target_size=(150,150))

Found 2500 images belonging to 2 classes.


In [29]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [36]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 batch_normalization_2 (Batc  (None, 72, 72, 64)       256       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                      

In [ ]:
history = model.fit(train_generator,epochs=3,validation_data=test_generator)

Epoch 1/3
237/352 [===================>..........] - ETA: 5:11 - loss: 0.4972 - accuracy: 0.7600

/home/milad/.local/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))


352/352 [==============================] - 977s 3s/step - loss: 0.4916 - accuracy: 0.7633 - val_loss: 0.5766 - val_accuracy: 0.6956
Epoch 2/3
352/352 [==============================] - 962s 3s/step - loss: 0.3969 - accuracy: 0.8215 - val_loss: 0.6226 - val_accuracy: 0.7020
Epoch 3/3
134/352 [==========>...................] - ETA: 10:21 - loss: 0.2650 - accuracy: 0.8920

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()
history_df.loc[:, ['accuracy', 'val_accuracy']].plot()
plt.show()

In [ ]:
test_image = load_img('1.jpg',target_size=(150,150))
plt.imshow(test_image)

test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
result = model.predict(test_image)
print("Dog") if(result>=0.5) else print("Cat")